<a href="https://colab.research.google.com/github/yueop/AS_LAB/blob/main/datasets_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')   #구글 드라이브 연결
Project_Path = '/content/drive/MyDrive/Colab Notebooks'  #저장할 폴더 경로 설정

#저장할 폴더 존재 여부 확인(없으면 새 폴더 생성, 있으면 기존 폴더 사용)
if not os.path.exists(Project_Path):
    os.makedirs(Project_Path)
    print(f"새 폴더 생성 후 저장 완료: {Project_Path}")
else:
    print(f"기존 폴더에 저장: {Project_Path}")

os.chdir(Project_Path)   #작업 위치 이동(change directory)

Mounted at /content/drive
기존 폴더에 저장: /content/drive/MyDrive/Colab Notebooks


In [ ]:
%%writefile my_datasets.py
import torch #일반적인 파이토치 기능(텐서 만들기 등)
from torch.utils.data import DataLoader, TensorDataset, random_split #데이터를 병합하고, 운송하고, 분할하는 기능
from torchvision import datasets, transforms #이미지를 모델이 알아볼 수 있게 변형하고, 대중적인 이미지 데이터셋들 가져오는 기능
from sklearn.datasets import load_iris #IRIS 데이터셋 로드
from sklearn.model_selection import train_test_split #train_dataset과 test_dataset을 랜덤하게 섞어 비율대로 잘라주는 기능
from sklearn.preprocessing import StandardScaler #IRIS 정규화 용도

class UnifiedDataLoader:
    def __init__(self, batch_size=64, aug_config=None):
        self.batch_size = batch_size
        self.aug_config = aug_config if aug_config else {'use_aug': False}

    def get_transform(self, dataset_name, mode):
        #IRIS는 변형 X
        if dataset_name == 'iris':
            return None

        #기본 변환(ToTensor, Nomalize)
        base_transforms = [
            transforms.ToTensor(), #이미지 데이터 텐서로 변환(전처리)
            transforms.Normalize((0.5,), (0.5,)) #데이터 정규화(-1.0~1.0 사이의 값으로 변환)
            ]

        if mode == 'train' and self.aug_config.get('use_aug', False): #학습 모드이면서 증강 설정이 True라면 증강 적용
            aug_list = []
            #회전
            if self.aug_config.get('rotation', 0) > 0:
                aug_list.append(transforms.RandomRotation(self.aug_config['rotation']))
            #좌우 반전
            if self.aug_config.get('flip_prob', 0) > 0:
                aug_list.append(transforms.RandomHorizontalFlip(p=self.aug_config['flip_prob']))
            #증강 -> 기본 변환 순서로 합치기
            return transforms.Compose(aug_list + base_transforms)

        else:
            #검증 / 테스트 할 때는 기본 변환만 적용
            return transforms.Compose(base_transforms)

    def get_loader(self, dataset_name, mode='train'):
        dataset_name = dataset_name.lower()
        if dataset_name == 'iris':
            return self.get_iris_loader(mode)
        elif dataset_name == 'mnist':
            return self.get_mnist_loader(mode)
        elif dataset_name == 'fashionmnist':
            return self.get_fashionmnist_loader(mode)
        else:
            print(f'지원하지 않는 데이터셋입니다.: {dataset_name}') #예외 처리
            return None

    def get_iris_loader(self, mode): #IRIS 데이터셋 로더
        iris = load_iris() #사이킷런 라이브러리에서 데이터 로드
        X, y = iris.data, iris.target

        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42) #데이터 분할

        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

        scaler = StandardScaler() #IRIS 데이터 정규화
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        X_test = scaler.transform(X_test)

        if mode == 'train':
            X_data, y_data = X_train, y_train
            shuffle = True
        elif mode == 'val':
            X_data, y_data = X_val, y_val
            shuffle = False
        else: #test
            X_data, y_data = X_test, y_test
            shuffle = False

        #텐서 변환
        X_tensor = torch.tensor(X_data, dtype=torch.float32)
        y_tensor = torch.tensor(y_data, dtype=torch.long)

        dataset = TensorDataset(X_tensor, y_tensor)
        return DataLoader(dataset, batch_size = self.batch_size, shuffle=shuffle)

    def get_image_dataset_split(self, dataset_class, dataset_name, mode):
        current_transform = self.get_transform(dataset_name, mode)
        if mode == 'test':
            dataset = dataset_class(root='./data', train=False, download=True, transform=current_transform)
            return DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

        full_train_dataset = dataset_class(root='./data', train=True, download=True, transform=current_transform) #훈련용 데이터 불러오기

        train_size = int(0.8 * len(full_train_dataset)) #학습용 데이터(80%)
        val_size = len(full_train_dataset) - train_size #검증용 데이터(20%)

        train_dataset, val_dataset = random_split(
            full_train_dataset, [train_size, val_size],
            generator = torch.Generator().manual_seed(42)
        )

        if mode == 'train':
            return DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        else: #valid
            return DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False)

    def get_mnist_loader(self, mode):
        return self.get_image_dataset_split(datasets.MNIST, 'mnist', mode)
    def get_fashionmnist_loader(self, mode):
        return self.get_image_dataset_split(datasets.FashionMNIST, 'fashionmnist', mode)

Overwriting my_datasets.py
